<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/clickbait_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
%load_ext autoreload
%autoreload 2

In [53]:
import sys
sys.path.append('/content/drive/MyDrive/NLP_Projects/custom_functions')

In [54]:
if 'google.colab' in str(get_ipython):
  from google.colab import drive
  drive.mount('/content/drive')
  !pip install -U swifter -qq
  !pip install -U nltk -qq
  !pip install -U spacy -qq
  !python -m spacy download en_core_web_sm -qq
  
  base_path = '/content/drive/MyDrive/NLP_Projects/Clickbait'
  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-25 02:11:37.584848: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-25 02:11:37.585028: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-25 02:11:37.585056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot d

In [55]:
from pathlib import Path
import pandas as pd
import spacy as sp
import swifter as sw

In [62]:
import custom_preprocessor_mod as cpm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
#import plot_learning_curve as plt

In [7]:
base_folder = Path(base_path)
data_folder = base_folder

In [8]:
for file in base_folder.iterdir():
  print(file)

/content/drive/MyDrive/NLP_Projects/Clickbait/clickbait_classification.ipynb
/content/drive/MyDrive/NLP_Projects/Clickbait/clickbait_data.csv


In [12]:
clickBait = pd.read_csv(data_folder/ "clickbait_data.csv",encoding = 'ISO-8859-1')
print(clickBait.shape)

(32000, 2)


In [13]:
print(clickBait.head(4))

                                            headline  clickbait
0                                 Should I Get Bings          1
1      Which TV Female Friend Group Do You Belong In          1
2  The New "Star Wars: The Force Awakens" Trailer...          1
3  This Vine Of New York On "Celebrity Big Brothe...          1


In [17]:
print(clickBait['headline'][100])

We Taught These Grandmas How To Do The Nae Nae And It Was Cute AF


In [18]:
trainData = clickBait.sample(frac=0.1, replace=True, random_state=1) #Get 10% of data as train dataset
testData = clickBait.sample(frac=0.1, replace=True, random_state=1) #Get 10% of data as test dataset
xtrain = clickBait['headline'].values  # extract "messages" values for x train variable
xtest = clickBait['headline'].values # extract "messages" values for x test variable
ytrain = clickBait['clickbait'].values
ytest = clickBait['clickbait'].values
print(f'X Train data:{xtrain.shape} Y Train data: {ytrain.shape}') # Display the train dataset's shape
print(f'X Test data: {xtest.shape} Y Test data: {ytest.shape}')

X Train data:(32000,) Y Train data: (32000,)
X Test data: (32000,) Y Test data: (32000,)


In [27]:
xtrain_preprocess = cpm.SpacyPreprocessor('en_core_web_sm').transform(xtrain)

/content/drive/MyDrive/NLP_Projects/custom_functions/custom_preprocessor_mod.py:84: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [35]:
tfidf_classifier = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='word', token_pattern=r"[\S]+")),
    ('classifier', LogisticRegression())
])

In [44]:
tfidf_param_grid = {'vectorizer__max_features': [100,500,1000], 'classifier__C': [0.01, 0.1, 0.5, 1]}

In [45]:
tfidf_grid_classifier = GridSearchCV(estimator=tfidf_classifier, param_grid=tfidf_param_grid, cv=3)
tfidf_grid_classifier.fit(xtrain_preprocess,ytrain)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(token_pattern='[\\S]+')),
                                       ('classifier', LogisticRegression())]),
             param_grid={'classifier__C': [0.01, 0.1, 0.5, 1],
                         'vectorizer__max_features': [100, 500, 1000]})

In [46]:
print(f'Best cross-validation score: {tfidf_grid_classifier.best_score_:.2f}')
print("\nBest parameters: ", tfidf_grid_classifier.best_params_)
print("\nBest estimator: ", tfidf_grid_classifier.best_estimator_)

Best cross-validation score: 0.92

Best parameters:  {'classifier__C': 1, 'vectorizer__max_features': 1000}

Best estimator:  Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=1000, token_pattern='[\\S]+')),
                ('classifier', LogisticRegression(C=1))])


In [60]:
#printed the results of best estimator score xtrain & ytain
print(tfidf_grid_classifier.score(xtrain_preprocess, ytrain))
print(tfidf_grid_classifier.best_score_)

0.926125
0.9162499773747812


In [63]:
tfidf_xtest_preprocess = cpm.SpacyPreprocessor(model='en_core_web_sm').transform(xtest)
tfidf_ytest_pred = tfidf_grid_classifier.predict(tfidf_xtest_preprocess)

print('\nTest set report: \n\n', 
      classification_report(ytest, tfidf_ytest_pred))

/content/drive/MyDrive/NLP_Projects/custom_functions/custom_preprocessor_mod.py:84: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")



Test set report: 

               precision    recall  f1-score   support

           0       0.91      0.95      0.93     16001
           1       0.94      0.91      0.92     15999

    accuracy                           0.93     32000
   macro avg       0.93      0.93      0.93     32000
weighted avg       0.93      0.93      0.93     32000

